In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
# Configure Azure storage access directly (no mount needed)
spark.conf.set("fs.azure.account.auth.type.tokyoolympicdatakasturi.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.tokyoolympicdatakasturi.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.tokyoolympicdatakasturi.dfs.core.windows.net", "**key**")
spark.conf.set("fs.azure.account.oauth2.client.secret.tokyoolympicdatakasturi.dfs.core.windows.net", "**key**")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.tokyoolympicdatakasturi.dfs.core.windows.net", "https://login.microsoftonline.com/**key**/oauth2/token")

# Now access files directly using the abfss:// path
# Example: df = spark.read.csv("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/your-file.csv***")

In [0]:
%fs
ls "abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net"

path,name,size,modificationTime
abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/raw-data/,raw-data/,0,1769991688000
abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/transformed-data/,transformed-data/,0,1769991700000


In [0]:
spark

In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/raw-data/teams.csv")
     

In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes.printSchema()
coaches.printSchema()
entriesgender.printSchema()
medals.printSchema()
teams.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesgender = entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  14|                Cuba|   7|     3|     5|   15|           18|
|  13|         New Zealand|   7|     6|     7|   20|          

In [0]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("TeamCountry","Gold").show()

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows


In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/transformed-data/athletes")

In [0]:
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://tokyo-olympic-data@tokyoolympicdatakasturi.dfs.core.windows.net/transformed-data/teams")